In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, QuantileTransformer, PowerTransformer, OrdinalEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NeighborhoodComponentsAnalysis, KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from xgboost import plot_importance

# import warnings
# warnings.filterwarnings('ignore')

**HELPERS**

### Загрузим данные

In [3]:
assessments = pd.read_csv("data/assessments.csv")
courses = pd.read_csv("data/courses.csv")
studentInfo = pd.read_csv("data/studentInfo.csv")
studentRegistration = pd.read_csv("data/studentRegistration.csv")
studentVle = pd.read_csv("data/studentVle.csv")
vle = pd.read_csv("data/vle.csv")
train_who = pd.read_csv("data/Train_Who.csv")
test_who = pd.read_csv("data/Test_Who.csv")
train_target_clf = pd.read_csv("data/Train_Target_clf.csv")

#### Склеим данные

In [4]:
# step 1
students_train = pd.merge(train_who, studentInfo, how="inner", on=["id_student", "code_module", "code_presentation"])
students_train.info()

assert(students_train.shape[0] == train_who.shape[0])
assert(
    (students_train["id_student"] == train_who["id_student"]).value_counts().all(axis=0) == True
) # Ок смержилось хорошо

# Есть немного пропусокв в признае imd_band

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    26074 non-null  int64 
 1   id_student            26074 non-null  int64 
 2   code_module           26074 non-null  object
 3   code_presentation     26074 non-null  object
 4   gender                26074 non-null  object
 5   region                26074 non-null  object
 6   highest_education     26074 non-null  object
 7   imd_band              25187 non-null  object
 8   age_band              26074 non-null  object
 9   num_of_prev_attempts  26074 non-null  int64 
 10  studied_credits       26074 non-null  int64 
 11  disability            26074 non-null  object
dtypes: int64(4), object(8)
memory usage: 2.4+ MB


In [132]:
# step 2
students_train2 = pd.merge(
    students_train,
    studentRegistration,
    how="inner",
    on=["id_student", "code_module", "code_presentation"]
)
students_train2.info()
assert(students_train2.shape[0] == train_who.shape[0])
assert(
    (students_train2["id_student"] == train_who["id_student"]).value_counts().all(axis=0)
) # Добавилось немного пропусков в date_registration

# step 3 - добавим целевую метку
students_train3 = pd.merge(
    students_train2,
    train_target_clf,
    how="inner",
    on="ID"
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    26074 non-null  int64  
 1   id_student            26074 non-null  int64  
 2   code_module           26074 non-null  object 
 3   code_presentation     26074 non-null  object 
 4   gender                26074 non-null  object 
 5   region                26074 non-null  object 
 6   highest_education     26074 non-null  object 
 7   imd_band              25187 non-null  object 
 8   age_band              26074 non-null  object 
 9   num_of_prev_attempts  26074 non-null  int64  
 10  studied_credits       26074 non-null  int64  
 11  disability            26074 non-null  object 
 12  date_registration     26037 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 2.6+ MB


In [153]:
# step 4 - общая часть для train/test
assessments_courses_info = pd.merge(
   assessments,
   courses,
   on=["code_module", "code_presentation"],
   how="inner"
)
assessments_courses_info = assessments_courses_info.rename({"date": "assessment_end_date"}, axis=1)
assert(assessments_courses_info.shape[0] == assessments.shape[0])

assessments_courses_info.info()
assessments_courses_info[assessments_courses_info["assessment_end_date"].isna()]
# это в основном экзамены. Потом с ними что то придумаем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 206 non-null    object 
 1   code_presentation           206 non-null    object 
 2   id_assessment               206 non-null    int64  
 3   assessment_type             206 non-null    object 
 4   assessment_end_date         195 non-null    float64
 5   weight                      206 non-null    float64
 6   module_presentation_length  206 non-null    int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 11.4+ KB


,code_module,code_presentation,id_assessment,assessment_type,assessment_end_date,weight,module_presentation_length
5,AAA,2013J,1757,Exam,NaN,100.0,268
11,AAA,2014J,1763,Exam,NaN,100.0,269
23,BBB,2013B,14990,Exam,NaN,100.0,240
35,BBB,2013J,15002,Exam,NaN,100.0,268
47,BBB,2014B,15014,Exam,NaN,100.0,234
53,BBB,2014J,15025,Exam,NaN,100.0,262
62,CCC,2014B,24290,Exam,NaN,100.0,241
63,CCC,2014B,40087,Exam,NaN,100.0,241
72,CCC,2014J,24299,Exam,NaN,100.0,269
73,CCC,2014J,40088,Exam,NaN,100.0,269


In [136]:
# step 5 - общая часть для train/test
print("studentVle count ", studentVle.shape[0])
for col in studentVle.columns:    
    print(f"nonna {col}: ", studentVle[col].count()) # Отлично пропусков нет

# Реализуем общую аггрегацию
grouping = studentVle.groupby(["id_student", "code_module", "code_presentation"], axis=0)
interaction_start = grouping["date"].min()
interaction_end = grouping["date"].max()
interaction_interval = (interaction_end - interaction_start + 1).reset_index()
interaction_end = interaction_end.reset_index().rename({"date": "date_interaction_end"}, axis=1)
interaction_interval = interaction_interval.rename({"date": "interaction_interval"}, axis=1)

sum_of_clicks = grouping["sum_click"].sum().reset_index()
sum_of_clicks = sum_of_clicks.rename({"sum_click": "sum_of_clicks"}, axis=1)

days_active = grouping.size().reset_index()
days_active = days_active.rename({0: "days_active"}, axis=1)

number_of_watched_sites = grouping["id_site"].nunique().reset_index()
number_of_watched_sites = number_of_watched_sites.rename({"id_site": "number_of_watched_sites"}, axis=1)

studentVle count  10655280
nonna code_module:  10655280
nonna code_presentation:  10655280
nonna id_student:  10655280
nonna id_site:  10655280
nonna date:  10655280
nonna sum_click:  10655280


#### Сгенерируем общую таблицу из аггрегации выше

In [138]:
# step 5 continuation
generated_features_p1 = pd.merge(
    interaction_interval,
    sum_of_clicks,
    on=["id_student", "code_module", "code_presentation"],
    how="inner"
)
assert(generated_features_p1.shape[0] == interaction_interval.shape[0] == sum_of_clicks.shape[0])
generated_features_p2 = pd.merge(
    generated_features_p1,
    interaction_end,
    on=["id_student", "code_module", "code_presentation"],
    how="inner"
)
assert(generated_features_p2.shape[0] == generated_features_p1.shape[0] == interaction_end.shape[0])
generated_features_p3 = pd.merge(
    days_active,
    number_of_watched_sites,
    on=["id_student", "code_module", "code_presentation"],
    how="inner"
)
assert(generated_features_p3.shape[0] == days_active.shape[0] == number_of_watched_sites.shape[0])

generated_features = pd.merge(
    generated_features_p2,
    generated_features_p3,
    on=["id_student", "code_module", "code_presentation"],
    how="inner"
)
assert(generated_features.shape[0] == generated_features_p2.shape[0] == generated_features_p3.shape[0])

generated_features.info() # все ок и даже пропусков нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29228 entries, 0 to 29227
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id_student               29228 non-null  int64 
 1   code_module              29228 non-null  object
 2   code_presentation        29228 non-null  object
 3   interaction_interval     29228 non-null  int64 
 4   sum_of_clicks            29228 non-null  int64 
 5   date_interaction_end     29228 non-null  int64 
 6   days_active              29228 non-null  int64 
 7   number_of_watched_sites  29228 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 1.8+ MB


In [113]:
# VLE выглядит крайне сомнительно - попробуем нужна ли эта таблица - не будем ее мержить
vle.sample(5)

,id_site,code_module,code_presentation,activity_type,week_from,week_to
3060,773122,DDD,2014B,subpage,NaN,NaN
3400,814000,DDD,2014J,subpage,NaN,NaN
1429,949688,BBB,2014J,url,NaN,NaN
1510,913691,BBB,2014J,subpage,NaN,NaN
2674,674391,DDD,2013J,url,NaN,NaN


In [157]:
students_train3["id_student"].nunique()

23579

In [163]:
# step 6 - объединим students_train + generated_features + assesments_courses
students_train_part1 = pd.merge(
    students_train3,
    generated_features,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)
# students_train_part1.info()
# students_train3
# generated_features
students_train_part1[students_train_part1["sum_of_clicks"].isna()]

,ID,id_student,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration,final_result,interaction_interval,sum_of_clicks,date_interaction_end,days_active,number_of_watched_sites
15,15,2587950,BBB,2013J,F,North Region,HE Qualification,0-10%,0-35,4,240,N,-57.0,0,NaN,NaN,NaN,NaN,NaN
27,27,601491,BBB,2013J,F,London Region,HE Qualification,30-40%,0-35,0,120,N,-58.0,0,NaN,NaN,NaN,NaN,NaN
40,40,611581,FFF,2014B,M,South West Region,A Level or Equivalent,10-20,0-35,0,60,Y,-89.0,0,NaN,NaN,NaN,NaN,NaN
45,45,650353,FFF,2014J,M,East Anglian Region,Lower Than A Level,80-90%,0-35,0,120,N,-149.0,0,NaN,NaN,NaN,NaN,NaN
56,56,608606,FFF,2013J,M,West Midlands Region,Lower Than A Level,90-100%,35-55,0,60,N,-35.0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26042,26042,2487785,BBB,2013J,F,Ireland,Lower Than A Level,80-90%,0-35,0,120,N,-141.0,0,NaN,NaN,NaN,NaN,NaN
26043,26043,2625752,BBB,2013J,F,East Anglian Region,Lower Than A Level,20-30%,0-35,0,180,N,-51.0,0,NaN,NaN,NaN,NaN,NaN
26052,26052,562267,BBB,2014B,M,North Western Region,Lower Than A Level,10-20,35-55,0,60,N,-24.0,0,NaN,NaN,NaN,NaN,NaN
26054,26054,548321,BBB,2013B,F,South West Region,A Level or Equivalent,30-40%,0-35,0,60,N,-79.0,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
studentVle[studentVle["id_student"]]